In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexing_distinct_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)

In [ ]:
def get_accuracy(idRankedTables, idQueryGoal):
    
    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable[0] == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [ ]:
indexing_distinct_tables.indices.close(index='distinct_tables')
indexing_distinct_tables.indices.put_settings(index='distinct_tables', body={"index": {"similarity": {"default": {"type": "BM25"}}}})
indexing_distinct_tables.indices.open(index='distinct_tables')

In [ ]:
def search_indexing(query,tableField):
    
    result= indexing_distinct_tables.search(
        index="distinct_tables", 
        body = {
        "_source": ["tablePgID"],
        "from" : 0,
        "size" : 300,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              #"fields": ["tablePgTitle"] 
              "fields": ["tablePgTitle","tablePgSummary"] 
              #"fields": [tableField]
            }
        }
    })
    
    return result

In [ ]:
articles = pd.read_csv('../dataset/data_articles_test.csv', delimiter=',')

In [ ]:
articles.head(1)

In [ ]:
def execute_search(tableField, topK):

    accuracy = []

    for i,row in (articles.iterrows()):

        articl_key = row['page_id']
        query = row['page_title']+" "+row['meta_description']

        result = search_indexing(query,tableField)

        if result['hits']['total'] > 0:

            return_tables = []

            for hit in result['hits']['hits']:

                tablePgId = hit['_source']['tablePgID']
                table_score = hit['_score']
                new_row = {"table_id": tablePgId,"table_score": table_score}
                return_tables.append(new_row)

            df_return_tables = pd.DataFrame(return_tables)
            df_return_tables_sorting = df_return_tables.sort_values('table_score', ascending=False)

            selected_top = df_return_tables_sorting.head(topK)
            min_score = selected_top['table_score'].min()
            draw_tables_socres = df_return_tables_sorting[df_return_tables_sorting['table_score'] >= min_score]
            final_ranked_tables = draw_tables_socres.iloc[:,0:1].values

            accuracy.append(get_accuracy(final_ranked_tables,articl_key))

    print(str(round(np.mean(accuracy),4)))

In [ ]:
tableField = ["tablePgTitle","tablePgFullText","tablePgSummary","tablePgKeywords","tableSectionTitle","tableCaption","tableHeader","tableBody"]
topK = [1,5,10,20,100]

In [ ]:
for k in topK:
    
    print("")
    
    for field in tableField:
        
        print("Acc@"+str(k)+" - "+field)
        
        execute_search(field,k)

In [ ]:
for k in topK:
    
    print("Acc@"+str(k))
    
    execute_search("",k)